In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os
%env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [2]:
path = %env folder

In [3]:
filename = r'raw_data\2024-07-28_ena_sequences.csv.gz'
chunksize = 100000  # Adjust this size based on your system's memory constraints

# Initialize a Counter to keep track of organism counts
rows = []
accessions = []
pmids = []
dois = []
pmcids = []
countries = []
coordinates = []
taxids = []
organisms = []
no_pubs = []

# Read the CSV in chunks
chunk_count = 0
for chunk in pd.read_csv(path + filename, compression='gzip', chunksize=chunksize, low_memory=False):
        if chunk_count in list(range(0,100000000, 1000000)):
            print(chunk_count)
        
        rows.append(len(list(chunk.index)))
    
        access = list(chunk['ACCESSION'])
        accessions.extend(access)
        
        pmid = list(chunk['PRIMARY_PMID'])
        pmids.extend(pmid)  
        
        doi = list(chunk['PRIMARY_DOI'])
        dois.extend(doi)  
        
        pmcid = list(chunk['PRIMARY_PMCID'])
        pmcids.extend(pmcid)
        
        taxid = list(chunk['TAXID'])
        taxids.extend(taxid)
        
        organism = list(chunk['ORGANISM'])
        organisms.extend(organism) 

        country = list(chunk['COUNTRY'])
        countries.extend(country)

        coordinates.extend(list(chunk['LAT_LON'][chunk['LAT_LON'].notna()]))
    
        no_pubs.extend(chunk['ACCESSION'][(chunk['PRIMARY_PMID'].isna()) & (chunk['PRIMARY_PMCID'].isna()) & (chunk['PRIMARY_DOI'].isna())])
    
    #if chunk_count == 2: 
    #    break
    
        chunk_count = chunk_count+chunksize
       


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000


In [4]:
#nr of unique taxIDs
len(set([int(x) for x in taxids if not pd.isna(x)]))

1532757

In [9]:
#create df
df =pd.DataFrame(zip(accessions, pmids, dois, pmcids, taxids, organisms, countries), columns = ['ACCESSION', 'PRIMARY_PMID', 'PRIMARY_DOI', 'PRIMARY_PMCID', 'TAXID', 'ORGANISM','COUNTRY'])

In [19]:
df[df['COUNTRY'] == 'Pacific Ocean']

,ACCESSION,PRIMARY_PMID,PRIMARY_DOI,PRIMARY_PMCID,TAXID,ORGANISM,COUNTRY
587,MW056645,NaN,NaN,NaN,1603213.0,Abyssorchomene distinctus,Pacific Ocean
714,KQ433663,NaN,NaN,NaN,37653.0,Octopus bimaculoides,Pacific Ocean
1221,DQ150635,NaN,NaN,NaN,77133.0,uncultured bacterium,Pacific Ocean
1386,HM858375,NaN,NaN,NaN,203449.0,uncultured marine eukaryote,Pacific Ocean
2683,FJ537315,19893617,10.1371/journal.pone.0007657,NaN,588879.0,uncultured marine chrysophyte,Pacific Ocean
...,...,...,...,...,...,...,...
36723213,OM782616,NaN,NaN,NaN,3029995.0,Haploniscus sp. SO TR-2023a,Pacific Ocean
36724029,OM782537,NaN,NaN,NaN,3029994.0,Haploniscus sp. KKT TR-2023a,Pacific Ocean
36724295,OM782629,NaN,NaN,NaN,3029996.0,Mastigoniscus aff. latus 1 TR-2023a,Pacific Ocean
36725163,OM782594,NaN,NaN,NaN,3029995.0,Haploniscus sp. SO TR-2023a,Pacific Ocean


In [8]:
# standardize country of origin
df_country2grp = pd.read_csv(path+R'raw_data\2024-08-25_country2grp.csv.gz',keep_default_na=False, na_values=[''])
df_country = pd.read_csv(path+R'raw_data\2024-08-25_country.csv.gz',keep_default_na=False, na_values=[''])


In [9]:
# standardize country of origin
df_country2grp = pd.read_csv(path+R'raw_data\2024-08-25_country2grp.csv.gz',keep_default_na=False, na_values=[''])
df_country = pd.read_csv(path+R'raw_data\2024-08-25_country.csv.gz',keep_default_na=False, na_values=[''])

#manually check that Taiwan is not linked to BRICS
df_country2grp

#rduce to main groups to simplify
main_groups = ['G77', 'OECD', 'BRICS']
df_country2grp_ed = df_country2grp.loc[df_country2grp['GRP'].isin(main_groups)]

#create dictionaries: (country name -->) iso --> group
country_iso = dict(zip(list(df_country['NAME']),list(df_country['ISO_STATE'])))
country_iso['Taiwan'] = 'TW'

iso_group_dict = dict(zip(list(df_country2grp_ed['ISO']),list(df_country2grp_ed['GRP'])))
iso_group_dict['IN'] = 'BRICS'
iso_group_dict['ZA'] = 'BRICS'
iso_group_dict['BR'] = 'BRICS'
iso_group_dict['CL'] = 'G77'
iso_group_dict['CO'] = 'G77'
iso_group_dict['CR'] = 'G77'
iso_group_dict['TW'] = np.NaN

df['ISO_ori'] = df['COUNTRY'].map(country_iso)
df['GROUP_ori'] = df['ISO_ori'].map(iso_group_dict)



In [10]:
len(df[df['COUNTRY'].notna()])

36770688

In [11]:
#here we mostly drop DSI with unclear origins
len(df[df['ISO_ori'].notna()])

36582148

In [12]:
#here we further drop cases where ISO is known but no group could be found
len(df[df['GROUP_ori'].notna()])

36002632

In [25]:
df[(df['ISO_ori'].notna()) & (df['GROUP_ori'].isna())]

,ACCESSION,PRIMARY_PMID,PRIMARY_DOI,PRIMARY_PMCID,TAXID,ORGANISM,COUNTRY,ISO_ori,GROUP_ori
3,KK913845,NaN,NaN,NaN,1472648.0,Aquilaria agallochum,Taiwan,TW,NaN
58,KF578407,NaN,NaN,NaN,36656.0,Penicillium crustosum,Antarctica,AQ,NaN
75,MK474327,NaN,NaN,NaN,2510848.0,Aliculastrum cylindricum,Palau,PW,NaN
81,KK312539,NaN,NaN,NaN,1447474.0,Mycobacterium tuberculosis MD17613,Moldova,MD,NaN
105,HM710879,21938020,NaN,NaN,77133.0,uncultured bacterium,Antarctica,AQ,NaN
...,...,...,...,...,...,...,...,...,...
36768148,ON005374,NaN,NaN,NaN,2697049.0,Severe acute respiratory syndrome coronavirus 2,Taiwan,TW,NaN
36768647,OM979071,NaN,NaN,NaN,293388.0,Bacillus aerius,Taiwan,TW,NaN
36769441,OM339444,NaN,NaN,NaN,1428.0,Bacillus thuringiensis,Uzbekistan,UZ,NaN
36769619,MT260765,NaN,NaN,NaN,111917.0,Lasiommata megera,Bulgaria,BG,NaN


In [13]:
df_wildsi_country = df[df['GROUP_ori'].notna()].reset_index(drop = True)

In [79]:
#export accessions numbers with known group IDs
df_wildsi_country.to_csv(path+R'processed_data/wildsi_country.csv')